In [1]:
import requests
import tensorflow.keras
from PIL import Image, ImageOps, UnidentifiedImageError
import numpy as np
import pandas as pd

CLASSES=["Cyclone","EarthQuake","Flood","WildFire"]

photo_dir=[]
photo_dir.clear()

def media_url(tweet):
    if "media" in tweet['entities']:
        for image in tweet['entities']['media']:
            return image['media_url']
    try:
        if "media" in tweet['retweeted_status']['entities']:
            for image in  tweet['retweeted_status']['entities']['media']:
                return image['media_url']
    except KeyError:
        return False
    else:
        return False

def CNN_classifier(tweet):
    link=media_url(tweet)
    if link==False:
        return False
    else:
        counter=0
        pic="disaster"+str(counter)+".png"
        counter=counter+1
        photoname="res/"+pic
        photo_dir.append(photoname)
        response = requests.get(link)
        file = open(photoname, "wb")
        file.write(response.content)
        file.close()
        np.set_printoptions(suppress=True)
        model = tensorflow.keras.models.load_model('keras_model.h5')
        data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
        problem_files=[]
        Value_err=[]
        try:
            for img in photo_dir:
                try:
                    image = Image.open(img).convert('RGB')
                except UnidentifiedImageError:
                    problem_files.append(img)
        except OSError as e:
            return False

        
        size = (224, 224)
        image = ImageOps.fit(image, size, Image.ANTIALIAS)
        image_array = np.asarray(image)
        normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
        try:
            data[0] = normalized_image_array
            #run the inference
            prediction = model.predict(data)
            lis=list(prediction[0])
            ind=lis.index(max(lis))
            pred_class=CLASSES[ind]
            return pred_class
        except ValueError:
            return False
        except TypeError:
            return False

In [2]:
from __future__ import print_function
import sys
from pyspark.streaming import StreamingContext
from pyspark import SparkContext,SparkConf
from pyspark.streaming.kafka import KafkaUtils
import json

import os
import sys
import ast
import json

import re
import string
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import threading
import time
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import pylab
from pylab import rcParams
import numpy as np
import multiprocessing

from pyspark import SparkContext
from pyspark import SQLContext,Row
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from pyspark.ml.feature import HashingTF,IDF, Tokenizer
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.clustering import StreamingKMeans
from pyspark.mllib.feature import StandardScaler
from pyspark.mllib.feature import Word2Vec
from pyspark.mllib.feature import Word2VecModel

BATCH_INTERVAL = 15  # How frequently to update (seconds)
clusterNum=15
# Change the backend to TkAgg to plot - only for Mac users
#plt.switch_backend('TkAgg')

def get_coord(post):
    coord = tuple()
    g = geocoder.geonames(post,key="ragulaaryan")
    try:
        if len(post)==0:
            coord = (0,0)
        else:
            coord = (float(g.lat),float(g.lng))
    except TypeError:
        #print ('error get_coord')
        coord=(0,0)
    return coord

def get_cordinates_from_event(text):
    chunked = ne_chunk(pos_tag(word_tokenize(text)))
    prev = None
    continuous_chunk = []
    current_chunk = []

    for subtree in chunked:
        if type(subtree) == Tree and subtree.label() == "GPE":
            current_chunk.append(" ".join([token for token, pos in subtree.leaves()]))
        if current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
        else:
            continue
            
    
    place=[]
    location=()
    if len(continuous_chunk)==0:
        location=(0,0)
        return location
    else:
        for item in range(len(continuous_chunk)):
            location=get_coord(continuous_chunk[item])
            return location
            break


def get_json_from_string(jsonString):
    '''
            Methos to convert the twitter feed event read from kafka to json
            Args:
                jsonString: even data as string
            Returns:
                json: converted json 
        '''
    
    try:
        json_object = json.loads(jsonString)
    except ValueError:
        return False
    return json_object

    
def doc2vec(twitter_feed_document):
    '''
            Method to get the vectors for every feed text from word2vec model already built using Word2Vec spark mllib library
            Args:
                twitter_feed_document: tuple of feed words got after pre processing (remove stop words, stemming)
            Returns:
                vecor representation of words in the feed text
        '''
    doc_vector = np.zeros(100)
    total_words = 0

    for word in twitter_feed_document:
        try:
            vector = np.array(word2vec_model_brooadcast.value.get(word))
            if vec!= None:
                doc_vector +=  vector
                total_words += 1
        except:
            continue

    if total_words>0:
        return doc_vector / float(total_words)
    return doc_vector


# Reg ex to remove special characters from feed text
special_character_regex = re.compile('[%s]' % re.escape(string.punctuation)) 
# list of stop words to remove, extracted from nltk python library 
stopwords=[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u'not', u'only', u'own', u'same', u'so', u'than', u'too', u'very', u's', u't', u'can', u'will', u'just', u'don', u'should', u'now']


def tokenize(text):
    '''
            Method to remove special characters, stop words from text and tokenize 
            Args:
                text: actual text to be tokenized
            Returns:
                list of tokens generated
        '''
    tokens = []
    #text = text.encode('ascii', 'ignore') #to decode
    text=re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*(),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text) # to replace url with ''
    text = special_character_regex.sub(" ",text)  # Remove special characters
    text=text.lower()

    for word in text.split():
        if word not in stopwords \
            and word not in string.punctuation \
            and len(word)>1 \
            and word != '``':
                tokens.append(word)
    return tokens

from collections import Counter
def get_most_popular_words(terms):
    '''
            Method to get the most most_common/ frequently used/polular words from the tweeet after clustering
            Args:
                terms: list of terms of clusters
            Returns:
                most top 5 frequently used words.
            We can also pass frequency as parameter 
        '''
    count_all = Counter()
    count_all.update(terms)
    return count_all.most_common(5)

def plot_data(q):
    '''
            Method to plot the data / cluster points after applying clustering on test data.
            This is the target method of python mulitprocessing 
            Args:
                q: A queue which contains the coordinates for plotting.
            Returns:
        '''
    import matplotlib.pyplot as plt
    from mpl_toolkits.basemap import Basemap
    import matplotlib.pyplot as plt
    from pylab import rcParams
    import numpy as np

    plt.ioff() 
    llon = -130
    ulon = 100
    llat = -30
    ulat = 60
    rcParams['figure.figsize'] = (14,10)
    my_map = Basemap(projection='merc',
                resolution = 'l', area_thresh = 1000.0,
                llcrnrlon=llon, llcrnrlat=llat, #min longitude (llcrnrlon) and latitude (llcrnrlat)
                urcrnrlon=ulon, urcrnrlat=ulat) #max longitude (urcrnrlon) and latitude (urcrnrlat)

    my_map.drawcoastlines()
    my_map.drawcountries()
    my_map.drawmapboundary()
    #my_map.fillcontinents(color = 'white', alpha = 0.3)
    #my_map.shadedrelief()
    #plt.pause(0.0001)
    #plt.show()


    colors = plt.get_cmap('jet')(np.linspace(0.0, 1.0, clusterNum))

    while True:
        if q.empty():
            time.sleep(5)

        else:
            # when queue is not empty plot the results
            obj=q.get()
            d=[x[0][0] for x in obj]
            c=[x[1] for x in obj]
            data = np.array(d)
            pcolor=np.array(c)
            try:
                xs,ys = my_map(data[:, 0], data[:, 1])
                my_map.scatter(xs, ys,  marker='o', alpha = 0.5,color=colors[pcolor])
                plt.pause(0.0001)
                plt.draw()
                plt.savefig("Clusters_plot_on_world_map.png")
                time.sleep(5)
            except IndexError: # Empty array
                pass


if __name__ == "__main__":
    '''
           Main method which does following
           1. Reads data from kafka topic - 'twitter-topic' in a BATCH_INTERVAL
           2. Reads the preivously build word2vec model using parquet
           3. Prepocesses(tokenize) feed data
           4. Prepares test and trainingData
           5. Runs streaming k-means model
           6. Clustering is based on geo location (using lat, long) and feeds text simliarity using word2vec.
           7. Plots the identified clusters and data using matplotlib. To achiece this I have used mulitprocessing with Queues as its a streaming system
           TODO: Persist the model to HDFS/ local files system for later use.
        '''
    # Multiprocessing
    q = multiprocessing.Queue()
    job_for_plotting = multiprocessing.Process(target=plot_data,args=(q,))
    job_for_plotting.daemon=True
    job_for_plotting.start()

    # Intialized Spark config
    #conf = SparkConf().setAppName("Kafka-Spark-Twitter-Feed-Clustering")
    #sc = SparkContext(conf=conf)
    sc = SparkContext.getOrCreate()
    sc.setLogLevel("ERROR")
    # get streaming context of batch interval 10 seconds
    #  TODO:
    #  Change this based on number of kakfka partitions , time to process current batch.
    # The above point is very crucial in prouction systems to achieve better paralelism in spark and handle backpresure
    stream=StreamingContext(sc,10) #
    
    # Read the stream into dstreams
    # Note : this is the loclahost mode
    Streaming = stream.socketTextStream("localhost", 9997)
    #print(kafkaStream.pprint())
    # Read word2vector model built offline using parquet
    sqlContext=SQLContext(sc)
    word2vec_model = sqlContext.read.parquet("resources/word2vector_model/data").alias("word2vec_model")
    #print(lookup.rdd.collectAsMap())
    print("Read Word2Vec model from parquet")
    # Boradcasting is used to avoid the copy of model in every machine/worker nodes
    word2vec_model_brooadcast = sc.broadcast(word2vec_model.rdd.collectAsMap())
    # The below code does folowing
    # Converts kafka event to json
    # Filters them based on created_at field in json. 
    # TODO: We can use Avro for schema management
    #Gets Coordinates lat,lng, and tokenize tweet text and append word2vect vector for every token
    print("Processing tweets....")
    processed_tweets= Streaming.map(lambda tweet : get_json_from_string(tweet)).filter(lambda tweet: tweet != False).filter(lambda tweet: 'created_at' in tweet).map(lambda tweet: (tweet["text"],CNN_classifier(tweet)))
    print("Processed tweets")
    processed_tweets.pprint()


Read Word2Vec model from parquet
Processing tweets....
Processed tweets


In [ ]:
stream.start()
stream.awaitTermination()

-------------------------------------------
Time: 2021-06-03 01:20:10
-------------------------------------------
('Winter Storm Windfall: Big Law Gets the Work to Defend Flood of Litigation https://t.co/nZXy0eFgv2 Texas LawyerAnge… https://t.co/wf7x8eAnwk', False)
('RT @RealBrysonGray: The rainbow is a representation of the promise that God made with Noah to never flood the earth again.', False)
('RT @szarng: swifties only talk about feminism when it comes to taylor', False)

-------------------------------------------
Time: 2021-06-03 01:20:20
-------------------------------------------
('Very important media training today for covering fires safely. Can never be too prepared. https://t.co/D5MrAM0ZJX', 'Flood')
('Absolutely delighted to be launching this new CPD series for @TheEPS1. Initially it’ll be monthly, but we may well… https://t.co/QGhcxCabUu', False)
('It’s a shame that God was creative and thoughtful enough to create the rainbow 🌈 as a covenant he wouldn’t flood us… https:/

-------------------------------------------
Time: 2021-06-03 01:22:00
-------------------------------------------
('New post (Gateway Of India Mumbai के पास की दीवारें व फुटपाथ पर Cyclone Tauktae की तबाही के निशान! |Hindi News) has… https://t.co/fjoeilAROy', False)
('RT @QuakesToday: 4.9 magnitude #earthquake. 126 km from #Hirara, #Okinawa, Japan https://t.co/1i9wi50lS9', False)
('RT @AssangeHome: Let\'s flood the UK GOVT with Julian Assange\'s "experience"...  \n\nFormal filings by .@DEAcampaign .@DefenseAssange .@Defend…', False)
('FLOOD ADVISORY now in effect for Golden Gate and Orangetree due to heavy rain being produced by this thunderstorm.… https://t.co/5p7DhhKS3c', False)
('The utter hypocrisy of this sh*t show of a government!! You could start  to #FreeJulianAssange who is incarcerated… https://t.co/OekSafOPqR', False)
('RT @TrinaSolis17: “The freight train of audits is traveling across the country!” @BorisEP \n“To all @CNN @MSNBC absolutely obsessed with thi…', False)

------

-------------------------------------------
Time: 2021-06-03 01:23:50
-------------------------------------------
('Cool, so within a decade there could be no more redwoods?', False)
('RT @QuakesToday: 3.0 magnitude #earthquake. 48 km from Pāhala, HI, #UnitedStates https://t.co/5dEN5CLgRM', False)
('#hashtag4 1pcs Ultrathin LED Flood light PIR Motion Sensor Detector waterproof Outdoor 85-265V IP65 Motion Sensor A… https://t.co/X3HPCNQjb0', False)

-------------------------------------------
Time: 2021-06-03 01:24:00
-------------------------------------------
('RT @TrinaSolis17: “The freight train of audits is traveling across the country!” @BorisEP \n“To all @CNN @MSNBC absolutely obsessed with thi…', False)
('wildfire by taeyeon thats it thats the tweet', False)

-------------------------------------------
Time: 2021-06-03 01:24:10
-------------------------------------------
('RT @ParksvilleNews: Prepare for the worst: 10 steps to get ready for wildfire smoke https://t.co/HwiszRc57P'